# Q* Learning with FrozenLake 4x4 

This Notebook implements an agent to learn FrozenLake, using Q Learning.</b>


![alt text](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

The goal of this game is <b>to go from the starting state (S) to the goal state (G)</b> by walking only on frozen tiles (F) and avoid holes (H). However, the ice is slippery, **so you won't always move in the direction you intend (stochastic environment)**

In [0]:
!pip install numpy
!pip install openai-gym
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

## Step 0: Import the dependencies 📚
We use 3 libraries:
- `Numpy` for our Qtable
- `OpenAI Gym` for our FrozenLake Environment
- `Random` to generate random numbers

In [0]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
import random

## Step 1: Create the environment 🎮
- Here we'll create the FrozenLake 8x8 environment. 
- OpenAI Gym is a library <b> composed of many environments that we can use to train our agents.</b>
- In our case we choose to use Frozen Lake.

In [0]:
env = gym.make("FrozenLake-v0")

## Step 2: Create the Q-table and initialize it 🗄️
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need, we need to calculate the action_size and the state_size
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [0]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [5]:
qtable = np.zeros((state_size, action_size))
print(qtable) #Will act as a cheat sheet of policies for our agent

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Tune the hyperparameters ⚙️


In [0]:
total_episodes = 20000        # Total episodes
learning_rate = 0.5           # Learning rate
max_steps = 100               # Max steps per episode
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001            # Minimum exploration probability 
decay_rate = 0.005             # Exponential decay rate for exploration prob

## Step 4: The Q learning algorithm 🧠

In [35]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: {}".format(str(sum(rewards)/total_episodes)))
print(qtable)

Score over time: 0.6852
[[5.33965258e-01 2.81702313e-01 3.83524125e-01 2.85587143e-01]
 [1.50048556e-01 2.71590306e-02 8.72494702e-02 4.55230406e-01]
 [2.37883060e-01 2.09458799e-01 1.70000485e-01 4.05241500e-01]
 [1.23239481e-01 1.26594053e-01 7.77405157e-02 3.55530447e-01]
 [5.67042307e-01 3.69497326e-01 2.54413729e-01 2.33804703e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.42743778e-01 1.97706757e-03 2.31186745e-03 3.59511209e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.38849031e-01 3.77562417e-01 1.25580878e-01 6.15744794e-01]
 [4.82917987e-02 7.32471852e-01 1.97727709e-01 1.76305069e-01]
 [6.37948394e-01 6.28746116e-02 7.36387536e-03 3.46278633e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.62456053e-01 3.21070703e-01 8.61392473e-01 2.45915037e-01]
 [5.25938643e-01 9.58935311e-01 4.81943470e-01 5.23380592e-01]
 [0.00000000e+00 0.00000000e+00

In [36]:
env.reset()

for episode in range(10):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 30
****************************************************
EPISODE  1
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 44
****************************************************
EPISODE  2
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 27
****************************************************
EPISODE  3
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 29
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 19
****************************************************
EPISODE  5
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 47
****************************************************
EPISODE  6
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 9
****************************************************
EPISODE  7
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 74
****************************************************
EPISODE  8
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of st

## Success Rate: 80%